In [12]:
! pip install transformers
! pip install fast_ml==3.68
! pip install datasets
! pip install accelerate -U
! pip install transformers[torch]
! pip install -U transformers
! pip install accelerate>=0.20.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
zsh:1: no matches found: transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
zsh:1: 0.20.1 not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import pandas as pd
import numpy as np
from transformers import Trainer, TrainingArguments, AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
import torch
import pathlib
from torch import nn
from torch.nn.functional import softmax

CURRENT_PATH = pathlib.Path().resolve()

In [14]:
DEVICE = torch.device('mps')
print (f'Device Availble: {DEVICE}')


Device Availble: mps


In [15]:
# Create a DataLoader class for processing and loading of the data during training and inference phase
class DataLoader(torch.utils.data.Dataset):
    def __init__(self, sentences=None, labels=None):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
        
        if bool(sentences):
            self.encodings = self.tokenizer(self.sentences,
                                            truncation = True,
                                            padding = True)
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        
        if self.labels == None:
            item['labels'] = None
        else:
            item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.sentences)
    
    
    def encode(self, x):
        return self.tokenizer(x, return_tensors = 'pt').to(DEVICE)


In [16]:
# SentimentModel class helps to initialize the model and contains the predict_proba and batch_predict_proba methods for single and batch prediction respectively. 
# The batch_predict_proba uses HuggingFace's Trainer to perform batch scoring
class SentimentModel():
    
    def __init__(self, model_path):
        
        self.model = AutoModelForSequenceClassification.from_pretrained(f'{CURRENT_PATH}/deep_learning/working').to(DEVICE)
        args =  TrainingArguments(output_dir=f'{CURRENT_PATH}/deep_learning/working/results', per_device_eval_batch_size=64)
        self.batch_model = Trainer(model = self.model, args= args)
        self.single_dataloader = DataLoader()
        
    def batch_predict_proba(self, x):
        
        predictions = self.batch_model.predict(DataLoader(x))
        logits = torch.from_numpy(predictions.predictions)
        proba = torch.nn.functional.softmax(logits, dim = 1).to('cpu').detach().numpy()
        return proba
        
        
    def predict_proba(self, x):
        x = self.single_dataloader.encode(x).to(DEVICE)
        predictions = self.model(**x)
        logits = predictions.logits
        proba = torch.nn.functional.softmax(logits, dim = 1).to('cpu').detach().numpy()
        return proba


In [17]:
sentiment_model = SentimentModel(f'{CURRENT_PATH}/deep_learning/working')
id2label = sentiment_model.model.config.id2label

/opt/anaconda3/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Inference Example

In [18]:
ARTICLE_TITLE = ''
print(f'{ARTICLE_TITLE[:50]}...: {id2label[np.argmax(sentiment_model.predict_proba(ARTICLE_TITLE))]}\n')

...: 8.0



## Testing

In [19]:
df = pd.read_csv(f'{CURRENT_PATH}/articles_and_labels.csv')
df_labels = df.loc[:, ['Article Title', 'Label']].dropna()
df_labels.head()

,Article Title,Label
0,NEW: Sam Altman has been pushed out of AI jugg...,8.0
1,The High Anxiety Of Holding On To Your Job And...,9.0
2,One of the richest women in the world Elisabet...,6.0
3,How Rob Gronkowski Is Thriving In Business Aft...,8.0
4,"While ""startup"" conjures up images of founders...",8.0


In [20]:
batch_sentences = df_labels.sample(n = 5, random_state = 1)['Article Title'].to_list()
single_sentence = df_labels.sample(n = 1, random_state = 2)['Article Title'].to_list()[0]


In [21]:
single_sentence_probas = sentiment_model.predict_proba(single_sentence)
predicted_class_label = id2label[np.argmax(single_sentence_probas)]

for article_title in df_labels['Article Title']:
    print(f'{article_title}: {id2label[np.argmax(sentiment_model.predict_proba(article_title))]}\n')

NEW: Sam Altman has been pushed out of AI juggernaut Open AI after an internal review found he had not been “consistently candid” with his representations to the board.Read the latest: trib.al/p83NQ63: -9.0

The High Anxiety Of Holding On To Your Job And The Great Apprehension Of Losing It trib.al/PTPG14u: -9.0

One of the richest women in the world Elisabeth DeLuca is worth an estimated $8.2 billion but she’s far from a household name and there are just a handful of photos of her online. trib.al/vDg0E1e: -9.0

How Rob Gronkowski Is Thriving In Business After The NFL trib.al/2PWwwph by @ScottKingMedia: -9.0

While "startup" conjures up images of founders in their twenties making a go of it in their college dorm rooms entrepreneurship isn’t just for the young. In fact the trend for older individuals to create and run a thriving business is accelerating. trib.al/MwrECVg: -9.0

One of the richest women in the world Elisabeth DeLuca is worth an estimated $8.2 billion but she’s far from a h

In [22]:
batch_sentence_probas = sentiment_model.batch_predict_proba(batch_sentences)
predicted_class_labels = [id2label[i] for i in np.argmax(batch_sentence_probas, axis = -1)]

100%|██████████| 1/1 [00:00<00:00, 1239.09it/s]
